In [189]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, SpectralEmbedding
from sklearn.preprocessing import MinMaxScaler

import model
from model import CalculateFeatures, SelectFeatures
%matplotlib qt


In [3]:
dtypes = {'Magnitude': np.float64, 'Latitude': np.float64, 'Longitude': np.float64, 'Depth': np.float64}
df = pd.read_csv('../../data/DATA_3_south.csv', delimiter=',', dtype=dtypes, parse_dates=["Datetime"],
                 date_parser=np.datetime64)
df = df.reset_index(drop=True)
df.head()

,Magnitude,Latitude,Longitude,Depth,Datetime
0,3.9,16.8023,-94.9130,114.9,2017-01-01 00:54:31
1,3.9,15.2842,-93.1210,100.7,2017-01-01 04:10:39
2,3.9,17.4503,-94.9543,138.3,2017-01-01 04:23:51
3,3.9,16.4077,-93.7075,139.4,2017-01-01 04:26:01
4,4.1,16.7017,-98.8633,27.5,2017-01-01 08:49:23


In [4]:
df_model = CalculateFeatures(df, 50, trim_features=True)

In [5]:
features = df_model.features.sort_values('nextDMaxMag')
features.head()

,old_index,firstT,lastT,meanMag,maxAMag,maxEMag,magDef,a,b,bStd,grcStd,elapsedT,rateSqrtEnergy,meanT,meanTStd,pMag6,zSeismicRateChange,bSeismicRateChange,last7dMaxMag,next14dMaxMag
31462,31534,2020-05-03 01:40:06,2020-05-05 08:47:56,3.902,4.2,4.407565,-0.207565,12.282871,2.786770,0.138838,42.284809,198470,173750.859086,4050.408163,1.588572e+09,1.902737e-17,109.316056,3.758304,4.6,4.4
31452,31524,2020-05-02 13:26:34,2020-05-05 01:54:29,3.886,4.4,4.300383,0.099617,14.564172,3.386715,0.349156,82.429952,217675,154469.238588,4442.346939,1.588528e+09,4.783105e-21,426.145515,-54.063364,4.6,4.4
31453,31525,2020-05-02 14:49:24,2020-05-05 01:54:48,3.892,4.4,4.333049,0.066951,13.772086,3.178382,0.302903,72.107358,212724,159858.836148,4341.306122,1.588532e+09,8.505671e-20,497.543968,-74.313965,4.6,4.4
31454,31526,2020-05-02 15:11:46,2020-05-05 02:54:24,3.892,4.4,4.333049,0.066951,13.772086,3.178382,0.302903,72.107358,214958,158197.466765,4386.897959,1.588536e+09,8.505671e-20,334.215499,33.909011,4.6,4.4
31455,31527,2020-05-02 15:30:39,2020-05-05 03:54:14,3.900,4.4,4.373212,0.026788,12.917771,2.953841,0.256418,61.514412,217415,158983.916427,4437.040816,1.588541e+09,1.892148e-18,350.499643,37.099205,4.6,4.4


In [25]:
X = features[model.FEATURES]
Y = features[model.TARGETS]
selected_features = SelectFeatures(features=X, targets=Y, corr_threshold=.95, fixed_features=["b"])

X=X[selected_features[0]].to_numpy()
Y=Y.to_numpy()

In [26]:
Y_Scaler = MinMaxScaler()
X_Scaler = MinMaxScaler()
X_norm64 = X_Scaler.fit_transform(X)
Y_norm64 = Y_Scaler.fit_transform(Y)
X_norm32 = X_norm64.astype(np.float32)
Y_norm32 = Y_norm64.astype(np.float32)

In [184]:
# evaluate isomap with logistic regression algorithm for classification
pca = PCA(n_components=2)
pca.fit(X_norm64.T, Y_norm64.T)
pca_components = pca.components_

NameError: name 'X_norm64' is not defined

In [185]:
cmap = cm.get_cmap("viridis")
colors = list(map(cmap, Y_norm64.T[0]))
sizes = np.power(10, Y_norm64.T[0] )

NameError: name 'Y_norm64' is not defined

In [186]:
%matplotlib qt
plt.scatter(pca_components[0], pca_components[1], s=sizes, color=colors)

In [31]:
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix = df_model.features.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()


In [39]:
embedding = Isomap(n_components=2)
X_transformed = embedding.fit_transform(X_norm32, Y_norm32)

%matplotlib qt
fix, ax = plt.subplots()
ax.set_facecolor((0, 0, .420, 0.125))
ax.scatter(X_transformed.T[0], X_transformed.T[1], s=sizes, color=colors)

In [183]:
import matplotlib.pyplot as plt
%matplotlib qt

fig, ax = plt.subplots()
num_ticks=np.round((Y_Scaler.data_max_-Y_Scaler.data_min_)*10+1).astype(int)[0]
labels=np.round(np.linspace(Y_Scaler.data_min_, Y_Scaler.data_max_, num=num_ticks).T[0], 1)
ticks=np.linspace(0, 1, num=num_ticks)[np.isin(labels,np.unique(Y))]
cax = plt.scatter(X_transformed.T[0], X_transformed.T[1], s=sizes, color=colors)

cbar = fig.colorbar(cax, ticks=ticks)
cbar.ax.set_yticklabels(np.unique(Y))
plt.plot()

[]

In [ ]:
%matplotlib qt
import seaborn as sns
import matplotlib.pyplot as plt

corrMatrix = features.corr()

sns.heatmap(corrMatrix, annot=True, cmap="bwr")
plt.show()


In [191]:
Embedding  = SpectralEmbedding(n_components=2)
Embedding.fit(X_norm32, Y_norm32)

SystemError: gstrf was called with invalid arguments